In [1]:
import numpy as np
import os, cv2
import h5py
import tifffile
from tqdm import tqdm, trange
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
from deepreg.predict import unwrapped_predict, normalize_batched_image
from deepreg.model.layer import Warping
import tensorflow as tf
import matplotlib.pyplot as plt


batch_size = 16
label_shape = (30, 3)

2025-03-20 15:17:00.393165: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-20 15:17:00.393208: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-20 15:17:00.394549: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Register a video in batches

In [ ]:
# tiff_path = "/store1/alicia/jellyfish/concat_cropped_0_11999.tif"
tiff_path = "/store1/alicia/jellyfish/concat_cropped_12000-29999.tif"

# checkpoint_path = "/home/brian/notebooks/brian/PBrainAlign_and_Jelly/private_BrainAlignNet/demo_notebook/long_first_jelly_again/save/ckpt-626"
# checkpoint_path = "/home/brian/notebooks/brian/PBrainAlign_and_Jelly/private_BrainAlignNet/demo_notebook/long_first_jelly_redo_DP/save/ckpt-396"
# checkpoint_path = "/home/brian/notebooks/brian/PBrainAlign_and_Jelly/private_BrainAlignNet/demo_notebook/long_first_jelly_redo_DP_real/save/ckpt-368"
# checkpoint_path = "/home/brian/notebooks/brian/PBrainAlign_and_Jelly/private_BrainAlignNet/demo_notebook/weights_shifted_2/save/ckpt-385"
# checkpoint_path = "/home/brian/notebooks/brian/PBrainAlign_and_Jelly/private_BrainAlignNet/demo_notebook/all_labs_flat-TOOMUCHNORM2/save/ckpt-29"
# checkpoint_path = "~/store1/brian/PBrainAlign_and_Jelly/private_BrainAlignNet/demo_notebook/all_labs_flat-TOOMUCHNORM2/save/ckpt-29"
# checkpoint_path = "/home/brian/notebooks/brian/PBrainAlign_and_Jelly/private_BrainAlignNet/demo_notebook/karen/save/ckpt-6"
checkpoint_path = "/home/brian/notebooks/brian/PBrainAlign_and_Jelly/private_BrainAlignNet/demo_notebook/karen_1_nolearn/save/ckpt-151"

output_video_path = "/home/brian/data4/brian/PBnJ/out_vids/no_align.mp4"
log_dir = "/home/brian/data4/brian/PBnJ/out_vids/logs"


In [3]:
inputs = tifffile.imread(tiff_path)

In [4]:
if 'batched_fixed_image' not in locals():
    fixed_image_ARCHIVE = inputs[0] # So we don't have to reload stuff
    batched_fixed_image = np.repeat(np.expand_dims(inputs[0], axis=0), batch_size, axis=0).astype(np.float32)
    print("Fixed img saved")
else:
    batched_fixed_image = np.repeat(np.expand_dims(fixed_image_ARCHIVE, axis=0), batch_size, axis=0).astype(np.float32)
    print("Restored")

Fixed img saved


In [5]:
os.chdir("/home/brian/notebooks/brian/PBrainAlign_and_Jelly/private_BrainAlignNet/scripts")
from register import set_GPU, register
# set_GPU(1)

In [6]:
# Green channel
green_channel_folder = "/home/brian/store1/brian/swiming_videos/GREEN CROPPED VIDEO FOR BRIAN"
green_files = sorted([f for f in os.listdir(green_channel_folder) if int(f.split('-')[0]) >= 12000], key=lambda x: int(x.split('-')[0])) # f.index("WARPED") < 0 and 
green_offsets = [int(f.split('-')[0]) - 12000 for f in green_files]

In [7]:
green_files

['12000-13999.tif - C=0.tif',
 '14000-15999.tif - C=0.tif',
 '16000-17999.tif - C=0.tif',
 '18000-19999.tif - C=0.tif',
 '20000-21999.tif - C=0.tif',
 '22000-23999.tif - C=0.tif',
 '24000-25999.tif - C=0.tif',
 '26000-27999.tif - C=0.tif',
 '28000-29999.tif - C=0.tif']

In [11]:
#### FIRST WE'RE GOING TO JUST TRY TO ALIGN TO A SINGLE FRAME
## The first batch of good videos was aligned using ckpt-90, going to switch to 626

# output_video_path = "/home/brian/data4/brian/PBnJ/out_vids/first_frame_align.mp4"
output_video_path = "/home/brian/data4/brian/PBnJ/out_vids/psuedo_col_control_ROIS_nowarp.mp4"
output_h5_path = "/home/brian/data4/brian/PBnJ/out_vids/first_frame_align.h5"
log_dir = "/home/brian/data4/brian/PBnJ/out_vids/logs"

config_path = os.path.join(checkpoint_path.split("/save")[0], "config.yaml")
model = None

max_frames = 400
# max_frames = 8
side_len = 1080
red_offset = 12000

padding = np.array([[0,0],[0,0],[0,0],[1,1]])

batched_fixed_image_pad = np.pad(batched_fixed_image[..., np.newaxis], padding, "constant", constant_values=0)

# warping = Warping(fixed_image_size=(side_len, side_len, 2), batch_size=batch_size)
warping = Warping(fixed_image_size=(side_len, side_len, 3), batch_size=batch_size, interpolation="nearest")

for file in green_files:
    green_in = tifffile.imread(os.path.join(green_channel_folder, file))
    outs = np.zeros_like(green_in)
    offset = int(file.split('-')[0])
    for frame in trange(offset, max_frames + offset, batch_size):
        if frame + batch_size > (max_frames + offset):
            frame = (max_frames + offset) - batch_size

        batched_moving_image = inputs[(frame - red_offset):(frame-red_offset)+batch_size].astype(np.float32)
        batched_moving_image = np.pad(batched_moving_image[..., np.newaxis], padding, "constant", constant_values=0)

        
        ddf_output, pred_fixed_image, model = unwrapped_predict(
            batched_fixed_image_pad,
            batched_moving_image,
            log_dir,
            label_shape,
            label_shape,
            model = model,
            model_ckpt_path = checkpoint_path,
            model_config_path = config_path,
        )

        gr_in = np.pad(green_in[(frame - offset):(frame - offset)+batch_size, ..., np.newaxis], padding, "constant", constant_values=0).astype(np.float32)
        outs[frame - offset:(frame - offset) + batch_size] = warping(inputs=[ddf_output, gr_in]).numpy()[..., 1]

    tifffile.imwrite(os.path.join(green_channel_folder, "WARPED_" + file), outs)
    break


  0%|          | 0/50 [00:00<?, ?it/s]

2025-03-19 21:08:42 | WARNING  | Log directory /home/brian/data4/brian/PBnJ/out_vids/logs exists already.
2025-03-19 21:08:42 | WARNING  | Using customized configuration. The code might break if the config doesn't match the saved model.
Built inputs.
Built control points.
Concatenated images.
{'extract_levels': ListWrapper([0, 1, 2, 3]), 'name': 'local', 'num_channel_initial': 16}
Built backbone.
Built DDF.
Built warping.


100%|██████████| 50/50 [02:11<00:00,  2.62s/it]
